In [3]:
def acked(err, msg):
    if err is not None:
        print("Failed to deliver message: %s: %s" % (str(msg.value()), str(err)))
    else:
        print("Message produced: %s" % (str(msg.value())))

In [ ]:
from socket import gethostname
from pandas import read_csv
from sys import exit
from confluent_kafka import Producer
TOPIC_NAME = "review_stream"
KAFKA_SERVER = "kafka:9092" 
CSV_FILE_PATH = "test_data.csv" 
DELAY_SECONDS = 1

# Các cột nhãn gốc từ file CSV
ASPECT_COLUMNS = ['Price', 'Shipping', 'Outlook', 'Quality', 'Size', 'Shop_Service', 'General', 'Others']

conf = {'bootstrap.servers': KAFKA_SERVER,
        'client.id': gethostname()}
producer = Producer(conf)

try:
    df = read_csv(CSV_FILE_PATH)
    required_columns = ['Review'] + ASPECT_COLUMNS
    missing_cols = [col for col in required_columns if col not in df.columns]
    if missing_cols:
        print(f"LỖI: Thiếu các cột sau trong {CSV_FILE_PATH}: {', '.join(missing_cols)}")
        exit(1)       
except FileNotFoundError:
    print(f"LỖI: Không tìm thấy file {CSV_FILE_PATH}")
    exit(1)
except Exception as e:
    print(f"Lỗi khi đọc CSV: {e}")
    exit(1)

Đang kết nối tới Kafka: kafka:9092
Đang đọc từ file: test_data.csv
Đang gửi tới topic: review_stream
Đã tải 2340 bản ghi từ file.


In [ ]:
from pandas import isna
from json import dumps
from time import sleep
try:
    for index, row in df.iterrows():
        review_text = row['Review']
        if isna(review_text):
            print("Bỏ qua bình luận rỗng (NaN)")
            continue
            
        payload = {
            "review_text": str(review_text)
        }
        for aspect in ASPECT_COLUMNS:
             label_value = row[aspect]
             if isna(label_value):
                 payload[aspect] = None
             else:
                 payload[aspect] = int(label_value)


        message_value = dumps(payload).encode('utf-8')
        
        producer.produce(TOPIC_NAME, value=message_value, callback=acked)
        
        producer.poll(0)
        
        # Đợi 1 giây
        print("By 23521198 - Lê Nguyễn Hoàng Phúc")
        sleep(DELAY_SECONDS)

except KeyboardInterrupt:
    print("Đã dừng bởi người dùng.")
finally:
    # Đảm bảo tất cả tin nhắn trong hàng đợi đều được gửi
    print("Đang xả (flushing) hàng đợi producer...")
    producer.flush()
    print("Hoàn tất.")

By 23521198 - Lê Nguyễn Hoàng Phúc
By 23521198 - Lê Nguyễn Hoàng Phúc
By 23521198 - Lê Nguyễn Hoàng Phúc
By 23521198 - Lê Nguyễn Hoàng Phúc
By 23521198 - Lê Nguyễn Hoàng Phúc
By 23521198 - Lê Nguyễn Hoàng Phúc
By 23521198 - Lê Nguyễn Hoàng Phúc
By 23521198 - Lê Nguyễn Hoàng Phúc
By 23521198 - Lê Nguyễn Hoàng Phúc
By 23521198 - Lê Nguyễn Hoàng Phúc
By 23521198 - Lê Nguyễn Hoàng Phúc
By 23521198 - Lê Nguyễn Hoàng Phúc
By 23521198 - Lê Nguyễn Hoàng Phúc
By 23521198 - Lê Nguyễn Hoàng Phúc
By 23521198 - Lê Nguyễn Hoàng Phúc
By 23521198 - Lê Nguyễn Hoàng Phúc
By 23521198 - Lê Nguyễn Hoàng Phúc
By 23521198 - Lê Nguyễn Hoàng Phúc
By 23521198 - Lê Nguyễn Hoàng Phúc
By 23521198 - Lê Nguyễn Hoàng Phúc
By 23521198 - Lê Nguyễn Hoàng Phúc
By 23521198 - Lê Nguyễn Hoàng Phúc
By 23521198 - Lê Nguyễn Hoàng Phúc
By 23521198 - Lê Nguyễn Hoàng Phúc
By 23521198 - Lê Nguyễn Hoàng Phúc
By 23521198 - Lê Nguyễn Hoàng Phúc
By 23521198 - Lê Nguyễn Hoàng Phúc
By 23521198 - Lê Nguyễn Hoàng Phúc
By 23521198 - Lê Ngu